# Chapter 5. Strictness and Laziness

- To say a function is non-strict means that the function may choose not to evluate one or more of its arguments.
- In contrast, a strict function always evaluates its arguments.

In [6]:
def square(x: Double): Double = x * x
square(15.0 + 1.0)
square(sys.error("failure"))

: 

Three examples of non-strict expressions in Scala that we are already know:

In [2]:
false && { println("!!"); true }
true || { println("!!"); false }
val result = if (input.isEmpty) sys.error("empty input") else input

res0_0: Boolean = false
res0_1: Boolean = true

How to lazily evaluate arguments in Scala:

In [3]:
def if2[A](cond: Boolean, onTrue: => A, onFalse: => A): A =
    if (cond) onTrue else onFalse

defined function if2

In [4]:
if2(false, sys.error("fail"), 3)

res2: Int = 3

In [5]:
if (3 < 0) sys.error("fail") else 3

res3: Int = 3

Caching the result of evaluating an argument:

In [18]:
def maybeTwice(b: Boolean, i: => Int) = if (b) i+i else 0

defined function maybeTwice

In [19]:
val x = maybeTwice(true, { println("hi") ; 1 + 41 })

hi
hi


x: Int = 84

In [22]:
def maybeTwice(b: Boolean, i: => Int) = {
    lazy val j = i
    if (b) j+j else 0
}

defined function maybeTwice

In [23]:
val x = maybeTwice(true, { println("hi") ; 1 + 41 })

hi


x: Int = 84

"We say that a non-strict function in Scala takes its arguments by name rather than by value"

In [48]:
def fib(n: Int): Int = 
    if (n == 0) 1
    else if (n == 1) 1
    else fib(n-1) + fib(n-2)

defined function fib

#####Lazy lists

In [8]:
sealed trait Stream[+A]
case object Empty extends Stream[Nothing]
case class Cons[+A](h: () => A, t: () => Stream[A]) extends Stream[A]

object Stream {
    def cons[A](hd: => A, tl: => Stream[A]): Stream[A] = {
        lazy val head = hd
        lazy val tail = tl
        Cons(() => head, () => tail)
    }

    def empty[A]: Stream[A] = Empty
    
    def apply[A](as: A*): Stream[A] = 
        if (as.isEmpty) empty else cons(as.head, apply(as.tail:_*))

}

defined trait Stream
defined object Empty
defined class Cons
defined object Stream

In [28]:
Stream(1,2,3)

res24: cmd24.INSTANCE.$ref$cmd23.Stream[Int] = Cons(<function0>,<function0>)

In [39]:
def headOption[A](str: Stream[A]): Option[A] = str match {
    case Empty => None
    case Cons(h, t) => Some(h())
}

defined function headOption

In [ ]:
val x = Cons(() => expensive(x), tl)
val h1 = x.headOption
val h2 = x.headOption

#### Separation of description and evaluation

In [9]:
def exists[A](str: Stream[A] ,p: A => Boolean): Boolean = str match {
    case Cons(h, t) => p(h()) || exists(t(),p)
    case _ => false
}

defined function exists

- Wrong code in the book?

In [6]:
def exists(p: A => Boolean): Boolean = this match {
    case Cons(h, t) => p(h()) || t().exists(p)
    case _ => false
}

: 

#### Foldright written to adopt laziness

In [10]:
def foldRight[A,B](str: Stream[A], z: => B)(f: (A, => B) => B): B = 
    str match {
        case Cons(h, t) => f(h(),foldRight(str,z)(f))
        case _ => z
}

defined function foldRight